In [1]:
# Import libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import collections
import pandas as pd
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# Load Fashion MNIST dataset from Keras
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, "y_test shape:", y_test.shape)

x_train shape: (60000, 28, 28) y_train shape: (60000,)
x_test shape: (10000, 28, 28) y_test shape: (10000,)


In [3]:
# Feature scaling: divide features by 255 to get a range of 0 to 1
x_train_scaled = x_train / 255.0
x_test_scaled = x_test / 255.0

# Convert y_train and y_test to one-hot vector
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Flatten training and test data
x_train_scaled_reshaped = np.reshape(x_train_scaled, 
                                     (x_train.shape[0], x_train.shape[1]*x_train.shape[2]))
x_test_scaled_reshaped = np.reshape(x_test_scaled, 
                                     (x_test.shape[0], x_test.shape[1]*x_test.shape[2]))

print(x_train_scaled.shape)
print(x_test_scaled.shape)
print(x_train_scaled_reshaped.shape)
print(x_test_scaled_reshaped.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 784)
(10000, 784)


In [0]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras import layers
from keras import optimizers

In [5]:
# Create base model of VGG19
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape = (100, 100, 3), classes = 10)

In [6]:
# Convert the data into 3 channels, as VGG19 requires input of shape (n, n, 3), where n can't be smaller than 48
X_train_3_channel = np.dstack([x_train_scaled_reshaped]*3)
X_test_3_channel = np.dstack([x_test_scaled_reshaped]*3)

print(X_train_3_channel.shape, X_test_3_channel.shape)

(60000, 784, 3) (10000, 784, 3)


In [7]:
# Convert to shape: (n, 28, 28, 3)
X_train_3_channel = X_train_3_channel.reshape(-1, 28, 28, 3)
X_test_3_channel = X_test_3_channel.reshape(-1, 28, 28, 3)

X_train_3_channel.shape, X_test_3_channel.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [8]:
# Resize the images as (100*100)
from keras.preprocessing.image import img_to_array, array_to_img

X_train_3_channel = np.asarray([img_to_array(array_to_img(im, scale=False).resize((100,100))) for im in X_train_3_channel])
X_test_3_channel = np.asarray([img_to_array(array_to_img(im, scale=False).resize((100,100))) for im in X_test_3_channel])

X_train_3_channel.shape, X_test_3_channel.shape

((60000, 100, 100, 3), (10000, 100, 100, 3))

In [9]:
# Split training set into training set(80%) and validation set(20%)
X_train_3_ch_sub, X_val_3_ch, y_train_3_ch_sub, y_val_3_ch = train_test_split(X_train_3_channel, 
                                                                              y_train_cat, 
                                                                              test_size=0.2, 
                                                                              random_state=66)

print(X_train_3_ch_sub.shape)
print(X_val_3_ch.shape)
print(X_test_3_channel.shape)

(48000, 100, 100, 3)
(12000, 100, 100, 3)
(10000, 100, 100, 3)


In [0]:
# Preprocess the input 
X_train = preprocess_input(X_train_3_ch_sub)
X_val = preprocess_input(X_val_3_ch)
X_test = preprocess_input(X_test_3_channel)

In [11]:
# Extract features
train_features = vgg19.predict(np.array(X_train), batch_size=128, verbose=1)
val_features = vgg19.predict(np.array(X_val), batch_size=128, verbose=1)
test_features = vgg19.predict(np.array(X_test), batch_size=128, verbose=1)

10000/10000 [==============================] - 22s 2ms/step


In [12]:
print(train_features.shape)

(48000, 3, 3, 512)


In [0]:
# Flatten extracted features
train_features = np.reshape(train_features, (48000, 3*3*512))
val_features = np.reshape(val_features, (12000, 3*3*512))
test_features = np.reshape(test_features, (10000, 3*3*512))

In [0]:
from keras.layers import LeakyReLU

# Add densely connected classifier followed by a LeakyReLU layer and final dense layer for the number of classes
model = Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(3*3*512)))
model.add(LeakyReLU(alpha=0.1))
model.add(layers.Dense(10, activation='softmax'))

In [0]:
# Compile
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Fit the model with extracted features

from keras import callbacks

reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=2,
    min_lr=0)

eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

callbacks = [reduce_learning, eary_stopping]

In [26]:
model.fit(train_features,
         y_train_3_ch_sub,
         batch_size=256,
         epochs=100,
         verbose=1,
         validation_data=(val_features, y_val_3_ch), 
         callbacks=callbacks)

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 4s 81us/step - loss: 3.5753 - acc: 0.1570 - val_loss: 3.5210 - val_acc: 0.2071
Epoch 2/100
48000/48000 [==============================] - 3s 67us/step - loss: 3.4375 - acc: 0.2039 - val_loss: 3.5100 - val_acc: 0.2008
Epoch 3/100
48000/48000 [==============================] - 3s 67us/step - loss: 2.6814 - acc: 0.2163 - val_loss: 2.0792 - val_acc: 0.2086
Epoch 4/100
48000/48000 [==============================] - 3s 68us/step - loss: 2.0747 - acc: 0.2297 - val_loss: 2.0129 - val_acc: 0.2432
Epoch 5/100
48000/48000 [==============================] - 3s 69us/step - loss: 2.0270 - acc: 0.2428 - val_loss: 1.9989 - val_acc: 0.2765
Epoch 6/100
48000/48000 [==============================] - 3s 67us/step - loss: 2.0024 - acc: 0.2514 - val_loss: 1.9632 - val_acc: 0.2819
Epoch 7/100
48000/48000 [==============================] - 3s 67us/step - loss: 1.9751 - acc: 0.2586 - val_loss: 1.9919 - 

In [27]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               2359808   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 2,364,938
Trainable params: 2,364,938
Non-trainable params: 0
_________________________________________________________________
